In [1]:
from experta import Fact, Rule, KnowledgeEngine, Field, DefFacts, AS, factlist, MATCH, L, W, P, TEST, AND, OR
import schema

In [2]:
class Animal(Fact):
    name = Field(str, mandatory=True)
    approx_weight = Field(int, mandatory=True)
#     approx_height = Field(int, mandatory=True)
    tail = Field(int, mandatory=True)
#     character = Field(str, mandatory=True)
    water = Field(bool, mandatory=True)
    related_animals = Field(list, mandatory=True)

In [3]:
class Person(Fact):
    name = Field(str, mandatory=False)
    animal_kind = Field(str, mandatory=True)
    age = Field(int, mandatory=True)
    friends = Field(list, mandatory=False)

In [4]:
class Lab(KnowledgeEngine):
    @Rule(Animal(water=True))
    def findTailWaterAnimal(self):
        print('Found tail water animal!')
    
        
    @Rule(Animal(related_animals=MATCH.related_animals))
    def findDogRelative(self, related_animals):
        if 'dog' in related_animals:
            print('Found dog relative!')
        

    @Rule(Person(friends=MATCH.friends))
    def findKittyFriends(self, friends):
        if 'Kitty' in friends:
            print('Find Kitty Friends')
            
    @Rule(Person(animal_kind=~L('cat'), name=MATCH.name, friends=MATCH.friends))
    def findAnimals(self, name, friends):
        print(f'{name} and his friend (not cat) {", ".join(list(friends))}')
        
    
    @Rule(AS.person << Person(name=P(lambda x: len(x) < 5), friends=P(lambda x: x[0] == "T")))
    def PredicateFindAnimals(self, person,  friends):
        for fr in friends:
            print(f'{person.as_dict()["name"]} and his friend (T...) {fr}' )
                    
    
    @Rule(Person(animal_kind=MATCH.animal_kind, age=MATCH.age),
          Animal(name=MATCH.name, approx_weight=MATCH.approx_weight),
          TEST(lambda name, animal_kind: animal_kind == name),
          TEST(lambda age, approx_weight: age < approx_weight / 5)
         )
    def f(self, name, age, animal_kind, approx_weight):
        print(name, animal_kind, age, approx_weight)


In [5]:
eng = Lab()

In [6]:
eng.reset()
eng.declare(Animal(name='dog', tail=1, water=False, approx_weight=20, approx_height=30, related_animals=['wolf']))
eng.declare(Animal(name='wolf', tail=1, water=False,  approx_weight=4, approx_height=10, related_animals=['dog', 'koyote']))
eng.declare(Animal(name='shark', tail=1, water=True, approx_weight=4, approx_height=40, related_animals=['fish']))
eng.facts

FactList([(0, InitialFact()),
          (1,
           Animal(name='dog', tail=1, water=False, approx_weight=20, approx_height=30, related_animals=frozenlist(['wolf',]))),
          (2,
           Animal(name='wolf', tail=1, water=False, approx_weight=4, approx_height=10, related_animals=frozenlist(['dog', 'koyote']))),
          (3,
           Animal(name='shark', tail=1, water=True, approx_weight=4, approx_height=40, related_animals=frozenlist(['fish',])))])

In [7]:
eng.run()

Found tail water animal!
Found dog relative!


In [8]:
eng.declare(Person(name="Rick", animal_kind='dog', age=3, friends=['Tim', 'Kitty']))
eng.declare(Person(name="Tim", animal_kind='cat', age=4, friends=['Rick']))
eng.declare(Person(name="Kitty", animal_kind='dog', age=4, friends=['Rick'])) 
eng.run()

Kitty and his friend (not cat) Rick
dog dog 3 20
Find Kitty Friends
Rick and his friend (not cat) Tim, Kitty


In [9]:
eng.facts.clear()
eng.facts

FactList()